In [1]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from rembg import remove

# denoise
from skimage import io
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import img_as_ubyte, img_as_float

from skimage.morphology import dilation
from skimage.morphology import rectangle
from skimage.measure import label, regionprops

In [2]:
image_path = "./images/"

images_list = os.listdir(image_path)

images = []
gray_images = []
for image in images_list:
    print(image)
    img = cv2.imread(image_path + image)
    resized = cv2.resize(img, (512, 512), interpolation=cv2.INTER_AREA)
    images.append(resized)
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    gray_images.append(gray)

print(images[0].shape)
print(gray_images[0].shape)

20230109_171910.jpg
20230109_171922.jpg
20230109_171941.jpg
20230109_171951.jpg
20230109_172007.jpg
20230109_172038.jpg
(512, 512, 3)
(512, 512)


In [3]:
def cv_seg(img):
    # ret, thresh = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    # ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_TRIANGLE)

    return thresh

def denoise(img):
    float_img = img_as_float(img)
    sigma_est = np.mean(estimate_sigma(img, channel_axis=-1))
    denoise_img = denoise_nl_means(float_img, h=1.15 * sigma_est, fast_mode=True, 
                               patch_size=5, patch_distance=3, channel_axis=-1)
    denoise_img_as_8byte = img_as_ubyte(denoise_img)
    # gray = cv2.cvtColor(denoise_img_as_8byte, cv2.COLOR_BGR2GRAY)
    return denoise_img_as_8byte

def multi_dil(im,num):
    for i in range(num):
        im = dilation(im)
    return im

## Image cropping

In [4]:
def crop(final_path, img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_AREA)
    rembg_img = remove(img)
    denoised = denoise(rembg_img)
    gray = cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY)
    segmented = cv_seg(gray)
    dilated = multi_dil(segmented, 3)
    label_im = label(segmented)
    for num, i in enumerate(regionprops(label_im)):
        minr, minc, maxr, maxc = i.bbox
        roi = img[minr:maxr, minc:maxc]
        cv2.imwrite(final_path + f"{num}.jpg", roi)

### crop base image

In [5]:
# crop("./base_img/", "./images/20230109_171941.jpg")

### crop test image

In [6]:
crop("./test_img/", "./images/20230109_171922.jpg")

### Opening cropped objects

In [7]:
# Opening objects
base_objects = os.listdir("./base_img/")
test_objects = os.listdir("./test_img/")

In [8]:
base_imgs = []
test_imgs = []

for obj in base_objects:
    base_imgs.append( cv2.imread("./base_img/" + obj))

for obj in test_objects:
    test_imgs.append( cv2.imread("./test_img/" + obj))

#### Naming objects for easier identification

In [9]:
test_pen = test_imgs[0]
test_botton = test_imgs[1]
# test_tape = test_imgs[0]
# test_drive = test_imgs[4]
# test_ears = test_imgs[2]
test_support = test_imgs[2]
test_cable = test_imgs[3]


In [10]:
base_pen = base_imgs[0]
base_botton = base_imgs[3]
base_tape = base_imgs[5]
base_drive = base_imgs[6]
base_ears = base_imgs[2]
base_support = base_imgs[4]
base_cable = base_imgs[1]

# test_pen = test_imgs[6]
# test_botton = test_imgs[3]
# test_tape = test_imgs[0]
# test_drive = test_imgs[4]
# test_ears = test_imgs[2]
# test_support = test_imgs[5]
# test_cable = test_imgs[1]

In [11]:
base_imgs = []
test_imgs = []

base_imgs = [base_pen,
base_botton, 
base_tape, 
base_drive,
base_ears,
base_support, 
base_cable ]

# test_imgs = [test_pen,
# test_botton,
# test_support,
# test_cable]

# test_imgs = [test_pen,
# test_botton,
# test_tape,
# test_drive,
# test_ears,
# test_support,
# test_cable]

In [12]:
def hist(img):
    # Calculate histogram without mask
    hist1 = cv2.calcHist([img],[0],None,[256],[0,256])
    hist2 = cv2.calcHist([img],[1],None,[256],[0,256])
    hist3 = cv2.calcHist([img],[2],None,[256],[0,256])
    # plt.subplot(221), plt.imshow(img)
    # plt.subplot(222), plt.plot(hist1), plt.plot(hist2),plt.plot(hist3)
    # plt.xlim([0,256])

    # plt.show()

    return np.array([np.squeeze(hist1), np.squeeze(hist2), np.squeeze(hist3)])

# def hist(img):
#     hist = cv2.calcHist([img], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
#     # hist = cv2.normalize(hist, hist).flatten()

#     plt.show()
#     return hist

In [13]:
test_tape_shape = test_tape.shape[0:2]
base_tape_shape = base_tape.shape[0:2]
base_support_shape = base_support.shape[0:2]

print(f"test tape = {test_tape_shape}")
print(f"base tape = {base_tape_shape}")
print(f"base tape = {base_support_shape}")

h_ratio = 1 - abs((test_tape_shape[0] - base_tape_shape[0]) / base_tape_shape[0])
w_ratio = 1 - abs((test_tape_shape[1] - base_tape_shape[1]) / base_tape_shape[1])

mean = (h_ratio + w_ratio) / 2
print(mean)

h_ratio = 1 - abs((test_tape_shape[0] - base_support_shape[0]) / base_support_shape[0])
w_ratio = 1 - abs((test_tape_shape[1] - base_support_shape[1]) / base_support_shape[1])

mean = (h_ratio + w_ratio) / 2
print(mean)




test tape = (25, 222)
base tape = (103, 131)
base tape = (48, 84)
0.2740309790261617
-0.06101190476190477


### Compairing shape

In [14]:
objs = ["pen", "botton", "tape", "drive", "ears", "support", "cable"]

for i in range(len(test_imgs)):
    test_shape = test_imgs[i].shape[:2]
    aux = []
    for j in range(len(base_imgs)):
        base_shape = base_imgs[j].shape[:2]
        height_ratio = 1 - abs((test_shape[0] - base_shape[0]) / base_shape[0])
        width_ratio  = 1 - abs((test_shape[1] - base_shape[1]) / base_shape[1])
        mean = (height_ratio + width_ratio) / 2
        aux.append(mean)
    maximum = max(aux)
    index = aux.index(maximum)
    match = objs[index]
    print(f"Match --> {match:8}: Distance = {maximum}")
    

Match --> drive   : Distance = 1.0
Match --> botton  : Distance = 1.0
Match --> tape    : Distance = 1.0
Match --> cable   : Distance = 1.0


## Custom chi-squared distance

In [15]:
def chi2_distance(histA, histB, eps = 1e-10):
	# compute the chi-squared distance
	d = 0.5 * np.sum([((a - b) ** 2) / (a + b + eps)
		for (a, b) in zip(histA, histB)])
	# return the chi-squared distance
	return d

In [16]:
objs = ["pen", "botton", "tape", "drive", "ears", "support", "cable"]

for i in range(len(base_imgs)):
    aux = []
    base_hist = hist(base_imgs[i])
    for j in range(len(test_imgs)):
        resized_test_img = cv2.resize(test_imgs[j], (base_imgs[i].shape[1], base_imgs[i].shape[0]), interpolation=cv2.INTER_AREA)
        test_hist = hist(resized_test_img)
        res = chi2_distance(base_hist, test_hist)
        aux.append(res)
    minimum = min(aux)
    index = aux.index(minimum)
    match = objs[index]
    print(f"{objs[i]:8} --> {match:8}: Distance = {minimum}")

pen      --> pen     : Distance = 3222.399169921875
botton   --> botton  : Distance = 0.0
tape     --> tape    : Distance = 0.0
drive    --> pen     : Distance = 0.0
ears     --> pen     : Distance = 2319.23388671875
support  --> tape    : Distance = 5248.10791015625
cable    --> drive   : Distance = 0.0


## SciPy distance metrics

In [17]:
from scipy.spatial import distance as dist

In [18]:
# initialize the scipy methods to compaute distances
SCIPY_METHODS = (
	("Euclidean", dist.euclidean),
	("Manhattan", dist.cityblock),
	("Chebysev", dist.chebyshev))


In [19]:
objs = ["pen", "botton", "tape", "drive", "ears", "support", "cable"]

for i in range(len(base_imgs)):
    aux = []
    base_hist = hist(base_imgs[i])
    for j in range(len(test_imgs)):
        resized_test_img = cv2.resize(test_imgs[j], (base_imgs[i].shape[1], base_imgs[i].shape[0]), interpolation=cv2.INTER_AREA)
        test_hist = hist(resized_test_img)
        res = dist.euclidean(base_hist.flatten(), test_hist.flatten())
        aux.append(res)
    minimum = min(aux)
    index = aux.index(minimum)
    match = objs[index]
    print(f"{objs[i]:8} --> {match:8}: Distance = {minimum}")

pen      --> botton  : Distance = 619.4497680664062
botton   --> botton  : Distance = 0.0
tape     --> tape    : Distance = 0.0
drive    --> pen     : Distance = 0.0
ears     --> botton  : Distance = 441.73748779296875
support  --> tape    : Distance = 1429.7965087890625
cable    --> drive   : Distance = 0.0


## OpenCV compareHist method

##### cv2.compareHist(H1, H2, method)  
**Methods**
- cv2.HISTCMP_CORREL: Computes the correlation between the two histograms.  
- cv2.HISTCMP_CHISQR: Applies the Chi-Squared distance to the histograms.  
- cv2.HISTCMP_INTERSECT: Calculates the intersection between two histograms.  
- cv2.HISTCMP_BHATTACHARYYA: Bhattacharyya distance, used to measure the “overlap” between the two histograms.  

In [20]:
# METHOD #1: UTILIZING OPENCV
# initialize OpenCV methods for histogram comparison
OPENCV_METHODS = (
	("Correlation", cv2.HISTCMP_CORREL),
	("Chi-Squared", cv2.HISTCMP_CHISQR),
	("Intersection", cv2.HISTCMP_INTERSECT),
	("Hellinger", cv2.HISTCMP_BHATTACHARYYA))
    
# loop over the comparison methods
for (methodName, method) in OPENCV_METHODS:
	# initialize the results dictionary and the sort
	# direction
	results = {}
	reverse = False
	# if we are using the correlation or intersection
	# method, then sort the results in reverse order
	if methodName in ("Correlation", "Intersection"):
		reverse = True

In [21]:
objs = ["pen", "botton", "tape", "drive", "ears", "support", "cable"]

for i in range(len(test_imgs)):
    aux = []
    base_hist = hist(base_imgs[i])
    for j in range(len(base_imgs)):
        resized_test_img = cv2.resize(test_imgs[j], (base_imgs[i].shape[1], base_imgs[i].shape[0]), interpolation=cv2.INTER_AREA)
        test_hist = hist(resized_test_img)
        res = cv2.compareHist(base_hist, test_hist, cv2.HISTCMP_BHATTACHARYYA) 
        aux.append(res)
    minimum = min(aux)
    index = aux.index(minimum)
    match = objs[index]
    print(f"{objs[i]:8} --> {match:8}: Distance = {minimum}")

IndexError: list index out of range

## RMSE method

In [ ]:
def rmse(A, B):
    return (((A - B) ** 2) ** (1/2)).mean()

#### RMSE in raw image

In [ ]:
objs = ["pen", "botton", "tape", "drive", "ears", "support", "cable"]

for i in range(len(base_imgs)):
    aux = []
    for j in range(len(test_imgs)):
        resized_test_img = cv2.resize(test_imgs[j], (base_imgs[i].shape[1], base_imgs[i].shape[0]), interpolation=cv2.INTER_AREA)
        res = rmse(base_imgs[i], resized_test_img)
        # print(f"{objs[i]} --> {objs[j]}: msre = {res}")
        aux.append(res)
    minimum = min(aux)
    index = aux.index(minimum)
    match = objs[index]
    print(f"{objs[i]:8} --> {match:8}: rmse = {minimum}")

pen      --> cable   : rmse = 8.46090933347273
botton   --> botton  : rmse = 7.0254464599356385
tape     --> tape    : rmse = 7.505118462601605
drive    --> ears    : rmse = 8.213454592328622
ears     --> ears    : rmse = 6.476090556032647
support  --> support : rmse = 5.997252541939605
cable    --> cable   : rmse = 6.176245321734958


#### RMSE between histograms

In [ ]:
objs = ["pen", "botton", "tape", "drive", "ears", "support", "cable"]

for i in range(len(base_imgs)):
    aux = []
    base_hist = hist(base_imgs[i])
    for j in range(len(test_imgs)):
        resized_test_img = cv2.resize(test_imgs[j], (base_imgs[i].shape[1], base_imgs[i].shape[0]), interpolation=cv2.INTER_AREA)
        test_hist = hist(resized_test_img)
        res = rmse(base_hist, test_hist)
        aux.append(res)
    minimum = min(aux)
    index = aux.index(minimum)
    match = objs[index]
    print(f"{objs[i]:8} --> {match:8}: rmse = {minimum}")


pen      --> ears    : rmse = 13.7109375
botton   --> botton  : rmse = 7.8046875
tape     --> tape    : rmse = 26.15104103088379
drive    --> ears    : rmse = 8.8359375
ears     --> ears    : rmse = 4.841145992279053
support  --> support : rmse = 13.84375
cable    --> cable   : rmse = 44.27604293823242


In [ ]:
def erase_dir(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
def match(base, test):
    base_histograms = []
    
    test_imgs = []


    for image in images_list:
        erase_dir(test_imgs)
        crop("./test_img/", "./images/" + image)
        test_objects = os.listdir("./test_img/")
        for obj in test_objects:
            test_imgs.append( cv2.imread("./test_img/" + obj))

        
        test_histograms = []

        
        for i in range(len(base_histograms)):
                
        base = hist(base)
    

    

IndentationError: expected an indented block after 'for' statement on line 18 (3559880687.py, line 20)

In [ ]:
base_histograms = []
for img in base_imgs:
    base_histograms.append(hist(img))

## Matching shape and histogram distances

In [24]:
objs = ["pen", "botton", "tape", "drive", "ears", "support", "cable"]
actual_img = []

for i in range(len(test_imgs)):
    aux = []
    test_shape = test_imgs[i].shape[:2]
    test_hist = hist(test_imgs[i])
    for j in range(len(base_imgs)):
        base_shape = base_imgs[j].shape[:2]
        height_ratio = 1 - abs((test_shape[0] - base_shape[0]) / base_shape[0])
        width_ratio  = 1 - abs((test_shape[1] - base_shape[1]) / base_shape[1])
        mean = (height_ratio + width_ratio) / 2
        if mean >= 0.75:
            resized_test_img = cv2.resize(test_imgs[i], (base_imgs[j].shape[1], base_imgs[j].shape[0]), interpolation=cv2.INTER_AREA)
            test_hist = hist(resized_test_img)
            res = cv2.compareHist(base_hist, test_hist, cv2.HISTCMP_BHATTACHARYYA) 
            aux.append(res)
        else:
            aux.append(1)
    minimum = min(aux)
    index = aux.index(minimum)
    match = objs[index]
    if minimum != 1:
        print(f"Match --> {match:8}: Distance = {minimum}")
        actual_img.append(match)

# missing = [element for element in objs if element not in actual_img]
print(f"missing objects: {list(element for element in objs if element not in actual_img)}")

Match --> ears    : Distance = 0.343463611455811
Match --> botton  : Distance = 0.4523030171106842
Match --> tape    : Distance = 0.40078003350806696
Match --> cable   : Distance = 0.4837320994273535
missing objects: ['pen', 'drive', 'support']
